In [ ]:
from tensorflow.keras.applications import EfficientNetV2L

model = EfficientNetV2L(weights='imagenet',
                        include_top=False,
                        input_shape=(224, 224, 3))

model.summary()

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

my_model = Sequential([model,
                       GlobalAveragePooling2D(),
                       Dense(512, activation='relu'),
                       Dropout(0.2),
                       Dense(512, activation='relu'),
                       Dropout(0.2),
                       Dense(1, activation='sigmoid')])

my_model.summary()

In [ ]:
import tensorflow as tf

BATCH_SIZE = 32
SEED = 1

train_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/dataset80-20/training',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(224, 224),
    shuffle=True,
    seed=SEED)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/dataset80-20/testing',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(224, 224),
    shuffle=False,
    seed=SEED
)

In [ ]:
iterator = iter(train_dataset)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(next(iterator)[0][0].numpy().astype(np.int32))

In [ ]:
from tensorflow.image import flip_left_right, adjust_brightness, adjust_contrast

def augment(image, label):
  image = flip_left_right(image)
  image = adjust_brightness(image, delta=0.1)
  image = adjust_contrast(image, contrast_factor=1.75)

  return (image, label)

train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

iterator = iter(train_dataset)

In [ ]:
plt.imshow(next(iterator)[0][0].numpy().astype(np.int32))

In [ ]:
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
train_dataset

In [ ]:
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset

In [ ]:
from tensorflow.keras.metrics import *

metrics = ['accuracy', Precision(), Recall(), AUC()]

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

my_model.compile(loss=BinaryCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, monitor='val_loss')

In [ ]:
my_model.fit(train_dataset, epochs=20,
             validation_data=val_dataset,
             callbacks=[es])

In [ ]:
file_path = '/content/drive/MyDrive/models 80-20/efficientnetv2_model-80-20.h5'

# Save the model as an HDF5 file
model.save(file_path)

print(f"EfficientNetV2 model saved as '{file_path}'.")

In [ ]:
my_model.save('model')

In [ ]:
from PIL import Image

image = Image.open('/content/drive/MyDrive/cnv sample/CNV-779442-1.jpeg')
image = image.resize((224, 224))
image

In [ ]:
img = np.asarray(image)

img.shape

In [ ]:
from copy import deepcopy

img_2 = deepcopy(img)

images = [img, img_2]

In [ ]:
prediction = my_model.predict(np.array(images))

prediction

In [ ]:
sorted(['cnv', 'normal'])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score, f1_score
from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/models 80-20/efficientnetv2_model-80-20.h5'
model = load_model(model_path)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

# Load your saved model
model_path = '/content/drive/MyDrive/models 80-20/efficientnetv2_model-80-20.h5'
loaded_model = tf.keras.models.load_model(model_path)

# Load your validation dataset (val_dataset) with labels
# You can use tfds.load or tf.keras.utils.image_dataset_from_directory to load the dataset

# Predict on the validation dataset
y_true = []
y_pred = []

for x, labels in val_dataset:
    y_true.extend(labels.numpy())
    predictions = loaded_model.predict(x)
    y_pred.extend((predictions > 0.5).astype(int).flatten())  # Assuming binary classification

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the confusion matrix
confusion = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion)

# Calculate the classification report
classification_rep = classification_report(y_true, y_pred, target_names=['cnv', 'normal'])

# Print the classification report
print("\nClassification Report:")
print(classification_rep)
